# CIMD Caching and Revocation

This notebook explores how Client-ID Metadata Documents (CIMD)
must be cached, invalidated, and controlled in real systems.

Goal:
- Understand why caching is required
- Understand why revocation is mandatory
- Learn where trust ends


## Why CIMD Metadata Must Be Cached

If metadata is fetched on every authorization request:

- Authorization Server becomes dependent on client uptime
- Attackers can cause metadata fetch floods (DoS)
- Latency becomes unpredictable

Therefore:
Caching is required for availability and stability.


## Why Metadata Cannot Be Trusted Forever

If metadata is cached forever:

- Client can be compromised
- Redirect URIs can change
- Keys can be rotated
- Permissions may be revoked

If cache never expires:
- Revocation is impossible
- Compromised clients remain valid



---

## 🔒 Cell 3 — Threat scenarios (Markdown)

```markdown
## Threat Scenarios Requiring Revocation

1. Client domain compromised
2. Malicious redirect URI added
3. Private key leaked
4. Agent behavior becomes unsafe
5. Client removed from allowlist

All require:
- Immediate or near-immediate revocation


In [2]:
import time

METADATA_CACHE = {}

CACHE_TTL_SECONDS = 600 # 10 minutes

def cache_metadata(client_id, meta_data):
    METADATA_CACHE[client_id] = {
        "meta_data" : meta_data,
        "expires_at" : time.time() + CACHE_TTL_SECONDS
    }
    
def get_cached_metadata(client_id):
    entry = METADATA_CACHE.get(client_id)
    if not entry:
        return None
    
    if time.time() > entry["expires_at"]:
        del METADATA_CACHE[client_id]
        return None
    
    return entry["metadata"]
    


In [3]:
# Simulated remote CIMD metadata documents
REMOTE_METADATA = {
    "https://agent.example.com/identity.json": {
        "client_id": "https://agent.example.com/identity.json",
        "redirect_uris": ["https://agent.example.com/callback"],
        "grant_types": ["authorization_code"],
        "scope": "research.read"
    }
}

AUTHORIZATION_CODES = {}
ISSUED_TOKENS = {}

In [ ]:
def fetch_cimd_with_cache(client_id):
    metadata = get_cached_metadata(client_id=client_id)
    
    if metadata:
        return metadata
    
    #simulated remote fetch
    metadata = REMOTE_METADATA.get(client_id)
    assert metadata, "metadata not found!"
    
    # Invariant: metadata must self-identify
    assert metadata["client_id"] == client_id
    
    cache_metadata(client_id, metadata)
    return metadata
    

## How Revocation Works in Practice

Revocation is NOT automatic.

It requires:
- Admin action
- Security automation
- Incident response

Common revocation controls:
- Manual cache invalidation
- Client denylist
- Forced re-fetch
- Reduced TTL during incident
